In [1]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
### HAMILTONIAN start
Molecule = 'LiH'
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., 1.45))]
basis = 'sto-3g'


### Get Hamiltonian
Hamilt = Hamiltonian(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='JW')
### HAMILTONIAN end

#####################################

Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(QubitHamiltonian)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)
anti_commuting_sets

Building Graph Edges: 100%|##########| 631/631 [00:17<00:00, 36.02it/s] 


{0: [(-4.087119674570706+0j) []],
 1: [(0.07823637778985248+0j) [Z8 Z9]],
 2: [(0.07823637778985218+0j) [Z6 Z7]],
 3: [(-0.22781796302888996+0j) [Z6],
  (-0.004217284878422751+0j) [Y6 Y7 X8 X9],
  (-0.0014751106104476744+0j) [Y0 Z1 Z2 Z3 Z4 Z5 Y6 X7 Z8 Z9 Z10 X11],
  (0.0034750158150631504+0j) [Y4 Z5 Y6 X7 Z8 Z9 Z10 X11],
  (0.004821390001231589+0j) [X6 Y7 Y10 X11],
  (0.004858871692969653+0j) [Y2 Z3 Z4 Z5 Y6 X7 Z8 Z9 Z10 X11]],
 4: [(-0.22781796302889024+0j) [Z9],
  (0.004217284878422751+0j) [X6 Y7 Y8 X9],
  (-0.001475110610447677+0j) [Y1 Z2 Z3 Z4 Z5 Z6 Z7 X8 X9 Y10],
  (-0.0024550578367610217+0j) [X0 X1 Y8 Y9],
  (0.002559511780295127+0j) [Y1 Z2 Z3 Y4 Y8 Y9],
  (0.0019065797888500565+0j) [Y1 Y2 Y8 Y9]],
 5: [(-0.22781796302889018+0j) [Z8],
  (0.004217284878422751+0j) [Y6 X7 X8 Y9],
  (0.0019065797888500567+0j) [Y0 Z1 Z2 Y3 X8 X9],
  (-0.001475110610447677+0j) [Y0 Z1 Z2 Z3 Z4 Z5 Z6 Z7 Y8 Y9 Z10 Y11],
  (-0.0024550578367610217+0j) [Y0 Y1 X8 X9],
  (0.0025595117802951267+0j) [Y0 Z1 Z2 Z

In [2]:
from quchem.Ansatz_Generator_Functions import *

##
NOON_spins_combined, NMO_basis = Hamilt.Get_NOON()
##
Hamilt.Get_CCSD_Amplitudes()



ansatz_obj = Ansatz(Hamilt.molecule.n_electrons, Hamilt.molecule.n_qubits)
reduced_Sec_Quant_CC_ops_ia, reduced_Sec_Quant_CC_ops_ijab, reduced_theta_parameters_ia, reduced_theta_parameters_ijab =ansatz_obj.Remove_NOON_terms(
    NOON=NOON_spins_combined,
    occ_threshold= 1.999,
    unocc_threshold=1e-4,
    indices_to_remove_list_manual=None, 
    single_cc_amplitudes=Hamilt.molecule.single_cc_amplitudes,
    double_cc_amplitudes=Hamilt.molecule.double_cc_amplitudes,
    singles_hamiltonian=Hamilt.singles_hamiltonian,
    doubles_hamiltonian=Hamilt.doubles_hamiltonian,
    tol_filter_small_terms=None)
reduced_Sec_Quant_CC_ops_ijab

[-1.0 [2^ 3^ 4 5] +
 1.0 [5^ 4^ 3 2],
 -1.0 [2^ 3^ 4 7] +
 1.0 [7^ 4^ 3 2],
 -1.0 [2^ 3^ 4 9] +
 1.0 [9^ 4^ 3 2],
 -1.0 [2^ 3^ 6 7] +
 1.0 [7^ 6^ 3 2],
 -1.0 [2^ 3^ 6 9] +
 1.0 [9^ 6^ 3 2],
 -1.0 [2^ 3^ 8 9] +
 1.0 [9^ 8^ 3 2]]

In [3]:
ia_terms, ijab_terms, ia_theta, ijab_theta = ansatz_obj.Get_ia_and_ijab_terms()

print('REDUCTION')
print('ia_terms', len(ia_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ia))
print('ijab_terms', len(ijab_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ijab))

REDUCTION
ia_terms 16 TO 6
ijab_terms 42 TO 6


In [4]:
Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab = ansatz_obj.UCCSD_single_trotter_step(reduced_Sec_Quant_CC_ops_ia,
                                                                                                                    reduced_Sec_Quant_CC_ops_ijab)

full_ansatz_Q_Circ = Ansatz_Circuit(Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab,
             Hamilt.molecule.n_qubits, Hamilt.molecule.n_electrons)

ansatz_cirq_circuit = full_ansatz_Q_Circ.Get_Full_HF_UCCSD_QC(reduced_theta_parameters_ia, reduced_theta_parameters_ijab)

print(ansatz_cirq_circuit.to_text_diagram(transpose=True)) 

0 1 2         3         4           5           6         7         8         9
│ │ │         │         │           │           │         │         │         │
X X X         X         H           │           H         │         H         H
│ │ │         │         │           │           │         │         │         │
│ │ Rx(0.5π)  │         │           │           │         │         │         │
│ │ │         │         │           │           │         │         │         │
│ │ @─────────X         │           │           │         │         │         │
│ │ │         │         │           │           │         │         │         │
│ │ │         @─────────X           │           │         │         │         │
│ │ │         │         │           │           │         │         │         │
│ │ │         │         Rz(-0.011π) │           │         │         │         │
│ │ │         │         │           │           │         │         │         │
│ │ │         @─────────X           │   

In [5]:

simulator = cirq.Simulator()
result = simulator.compute_amplitudes(ansatz_cirq_circuit, bitstrings=[i for i in range(2 ** len(ansatz_cirq_circuit.all_qubits()))])
result=np.around(result, 5)
print(result.reshape([(2 ** len(ansatz_cirq_circuit.all_qubits())), 1]))

[[ 0.+0.j]
 [ 0.+0.j]
 [-0.+0.j]
 ...
 [ 0.+0.j]
 [-0.+0.j]
 [-0.+0.j]]


In [6]:
from quchem.LCU_method import *

In [7]:
from quchem.Simulating_Quantum_Circuit import *
R_uncorrected, Pn, gamma_l = Get_R_linear_combination(anti_commuting_sets[10], 1)
R_corrected_Op_list, R_corr_list, ancilla_amplitudes, l1 = absorb_complex_phases(R_uncorrected)

full_circ = Full_Q_Circuit(Pn, R_corrected_Op_list, R_corr_list, ancilla_amplitudes, Hamilt.molecule.n_qubits, ansatz_cirq_circuit)
output_bin_dict = Get_binary_dict_project(full_circ, Pn, 100000, Hamilt.molecule.n_qubits, ancilla_amplitudes, l1)
expectation_value_by_parity(output_bin_dict) * gamma_l 

(0.06979766927711396+0j)

In [ ]:
# from timeit import default_timer as timer
# n_shots = 1000

# start = timer()
# simulator = cirq.Simulator()
# raw_result = simulator.run(full_circ, repetitions=10*n_shots * int(np.ceil(1 / (1/l1)**2)))
# end = timer()

# print('time taken = ', end - start ,'seconds')

In [8]:
from timeit import default_timer as timer

n_shots=100

###
start = timer()

testing =  VQE_Experiment_LCU_UP(anti_commuting_sets,
                 ansatz_cirq_circuit,
                 n_shots,
                 Hamilt.molecule.n_qubits,
                 N_indices_dict=None)
print(testing.Calc_Energy())

end = timer()
###

print('time taken = ', end - start ,'seconds')

time taken =  26.926637019001646 seconds


In [9]:
Hamilt.molecule.fci_energy

array(-7.88098231)

In [10]:
n_shots=100

def GIVE_ENERGY(theta_ia_theta_jab_list):
    theta_ia = theta_ia_theta_jab_list[:len(reduced_theta_parameters_ia)]
    theta_ijab = theta_ia_theta_jab_list[len(reduced_theta_parameters_ia):]
   
    ansatz_cirq_circuit = full_ansatz_Q_Circ.Get_Full_HF_UCCSD_QC(theta_ia, theta_ijab)

    VQE_exp_LCU =  VQE_Experiment_LCU_UP(anti_commuting_sets,
                     ansatz_cirq_circuit,
                     n_shots,
                     Hamilt.molecule.n_qubits,
                     N_indices_dict=None)#{7:0, 8:1, 9:0, 10:1})
    return VQE_exp_LCU.Calc_Energy().real


In [ ]:
### optimizer
from quchem.Scipy_Optimizer import *
THETA_params=[0 for _ in range(len(reduced_theta_parameters_ia)+len(reduced_theta_parameters_ijab))]

GG = Optimizer(GIVE_ENERGY, THETA_params, 'Nelder-Mead', store_values=True, display_iter_steps=True,
               tol=1e-5,
               display_convergence_message=True)
GG.get_env(50)
GG.plot_convergence()
plt.show()

### Tensorflow

In [ ]:
from quchem.TensorFlow_Opt import *

**gradient is given by**

https://arxiv.org/pdf/1906.08728.pdf

$$\frac{\partial O(\theta)}{\partial \theta}=\left\langle\overrightarrow{0}\left|\hat{U}^{\dagger} \hat{R}_{y}^{C \dagger}(\theta+\pi / 4) \hat{V}^{\dagger} \hat{O} \hat{V} \hat{R}_{y}^{C}(\theta+\pi / 4) \hat{U}\right| \overrightarrow{0}\right\rangle -\left\langle\overrightarrow{0}\left|\hat{U}^{\dagger} \hat{R}_{y}^{C \dagger}(\theta-\pi / 4) \hat{V}^{\dagger} \hat{O} \hat{V} \hat{R}_{y}^{C}(\theta-\pi / 4) \hat{U}\right| \overrightarrow{0}\right\rangle$$

$$\frac{\partial O(\theta)}{\partial \theta} =O(\theta+\pi / 4)-O(\theta-\pi / 4)$$

In [ ]:
def calc_gradient(theta_ia_theta_jab_list):
    
    grad_list=[]
    
    for index, theta in enumerate(theta_ia_theta_jab_list):
        
        new_theta_list = theta_ia_theta_jab_list.copy()
        new_theta_list[index] = theta + np.pi/4
        
        Obs_PLUS = GIVE_ENERGY(new_theta_list)
        
        
        new_theta_list[index] = theta - np.pi/4
        Obs_MINUS = GIVE_ENERGY(new_theta_list)
        
        gradient = Obs_PLUS - Obs_MINUS
        
        grad_list.append((gradient, theta))
        

    return grad_list

note:
this is very SLOW as it has to run a separate experiment TWICE for each parameter before taking a step!

In [ ]:
import random

X0 = [random.uniform(0, 2*np.pi) for _ in range(len(reduced_Sec_Quant_CC_ops_ia) + len(reduced_Sec_Quant_CC_ops_ijab))]

GG = Tensor_Flow_Optimizer(GIVE_ENERGY, X0, 'Adam', calc_gradient, learning_rate=0.1, beta1=0.9,
                            beta2=0.999, store_values=True, display_iter_steps=True)
GG.optimize(50)
GG.plot_convergence()

In [11]:
from quchem.Adam_Optimizer import *
import random


In [12]:
def calc_gradient_ADAM(theta_ia_theta_jab_list):
    
    grad_list=[]
    
    for index, theta in enumerate(theta_ia_theta_jab_list):
        
        new_theta_list = theta_ia_theta_jab_list.copy()
        new_theta_list[index] = theta + np.pi/4
        
        Obs_PLUS = GIVE_ENERGY(new_theta_list)
               
        
        new_theta_list[index] = theta - np.pi/4
        Obs_MINUS = GIVE_ENERGY(new_theta_list)
        
        gradient = Obs_PLUS - Obs_MINUS
        
        grad_list.append(gradient)
        

    return np.array(grad_list)

In [13]:
X0 = np.array([random.uniform(0, 2*np.pi) for _ in range(len(reduced_Sec_Quant_CC_ops_ia) + len(reduced_Sec_Quant_CC_ops_ijab))])

opt_params, list_of_inputs, list_of_outputs = Adam_Opt(X0, GIVE_ENERGY,
                                                       calc_gradient_ADAM, 
                                                       learning_rate=0.1, 
                                                       beta_1=0.9, 
                                                       beta_2=0.999, 
                                                       epsilon=1e-8,
                                                       max_iter=50,
                                                       disp=True,
                                                       tolerance=1e-3,
                                                       store_steps=True)

VQE_experiment_ENERGY(opt_params)

KeyboardInterrupt: 

(4096, 4096)

In [20]:
def LinAlgEnergy(theta_ia_theta_jab_list):
                 
    theta_ia = theta_ia_theta_jab_list[:len(reduced_theta_parameters_ia)]
    theta_ijab = theta_ia_theta_jab_list[len(reduced_theta_parameters_ia):]

    Qubit_Ham_matrix_JW = Hamilt.Get_sparse_Qubit_Hamiltonian_matrix(QubitHamiltonian)

    ansatz_lin_alg_obj = Ansatz_MATRIX(Hamilt.molecule.n_electrons, Hamilt.molecule.n_qubits, reduced_Sec_Quant_CC_ops_ia, reduced_Sec_Quant_CC_ops_ijab)
    # state_ket = ansatz_lin_alg_obj.Calc_ansatz_state_withOUT_trot(reduced_theta_parameters_ia, reduced_theta_parameters_ijab, 'JW')

    state_ket = ansatz_lin_alg_obj.Calc_ansatz_state_WITH_trot_SINGLE_STEP(theta_ia, theta_ijab, 'JW')

    Energy = ansatz_lin_alg_obj.Calc_energy_of_state(state_ket, Qubit_Ham_matrix_JW)

    return Energy

X0 = np.array([random.uniform(0, 2*np.pi) for _ in range(len(reduced_Sec_Quant_CC_ops_ia) + len(reduced_Sec_Quant_CC_ops_ijab))])
LinAlgEnergy(X0)

-7.309502118988392

In [22]:
def calc_gradient_ADAM_linAlg(theta_ia_theta_jab_list):
    
    grad_list=[]
    
    for index, theta in enumerate(theta_ia_theta_jab_list):
        
        new_theta_list = theta_ia_theta_jab_list.copy()
        new_theta_list[index] = theta + np.pi/4
        
        Obs_PLUS = LinAlgEnergy(new_theta_list)
               
        
        new_theta_list[index] = theta - np.pi/4
        Obs_MINUS = LinAlgEnergy(new_theta_list)
        
        gradient = Obs_PLUS - Obs_MINUS
        
        grad_list.append(gradient)
        

    return np.array(grad_list)
calc_gradient_ADAM_linAlg(X0)

array([ 0.01475187,  0.00455374, -0.00417454,  0.11173833,  0.38320635,
       -0.03372764,  0.0137972 , -0.00105375, -0.00050812, -0.00252228,
       -0.00399949, -0.00085024])

In [23]:
opt_params, list_of_inputs, list_of_outputs = Adam_Opt(X0, LinAlgEnergy,
                                                       calc_gradient_ADAM_linAlg, 
                                                       learning_rate=0.1, 
                                                       beta_1=0.9, 
                                                       beta_2=0.999, 
                                                       epsilon=1e-8,
                                                       max_iter=50,
                                                       disp=True,
                                                       tolerance=1e-3,
                                                       store_steps=True)

LinAlgEnergy(opt_params)

step: 1 input:[5.04213308 4.14366431 1.41376791 3.23940795 0.95961476 3.08224714
 2.40115544 6.24775264 4.86745981 3.67610006 1.71695519 5.18690979] obj_funct: -7.368382256364635
step: 2 input:[4.94246507 4.05868458 1.51386478 3.14042239 0.85950417 3.18004039
 2.30540359 6.33316299 4.9596402  3.77393877 1.81709048 5.27279217] obj_funct: -7.425774752661142
step: 3 input:[4.84731292 4.01161695 1.61031062 3.04774133 0.75916116 3.24573314
 2.2305696  6.4228554  5.05228727 3.87284611 1.91744543 5.32900916] obj_funct: -7.4755351494338855
step: 4 input:[4.77189131 3.99938056 1.69440587 2.97631293 0.65864076 3.24334393
 2.20072925 6.51628614 5.1092366  3.97081151 2.01741465 5.36356929] obj_funct: -7.520131290386636
step: 5 input:[4.73465645 4.00324898 1.76255388 2.94315329 0.55816331 3.20696846
 2.21415233 6.61204725 5.12876784 4.06219615 2.11272619 5.38293199] obj_funct: -7.5631303684641855
step: 6 input:[4.73059119 4.01226813 1.81801379 2.94638428 0.45809582 3.1580138
 2.25503453 6.70864877 

step: 46 input:[4.48785477 4.29734839 1.97287305 3.11650842 0.25582945 1.97184035
 3.79966762 7.75911063 4.47976699 6.23316136 3.36383623 6.19642258] obj_funct: -7.8338861221247145
step: 47 input:[4.48000172 4.3048794  1.90410425 3.12995799 0.2597164  1.83183985
 3.83122453 7.75700897 4.42244717 6.25138077 3.33725201 6.24789648] obj_funct: -7.850114421462142
step: 48 input:[4.50500044 4.33547154 1.83952389 3.13907055 0.26367761 1.69271618
 3.85986496 7.7677638  4.37801874 6.29563313 3.33258077 6.2586652 ] obj_funct: -7.85819469219815
step: 49 input:[4.53102019 4.38602217 1.78183595 3.14406616 0.27109034 1.55982049
 3.88595295 7.79779357 4.36903268 6.34001722 3.3195179  6.2477586 ] obj_funct: -7.859078113533457
step: 50 input:[4.52776195 4.42573835 1.72722763 3.14817708 0.28249851 1.43945444
 3.90949061 7.84202741 4.38961051 6.35661061 3.28588646 6.25964932] obj_funct: -7.858084419111499


NameError: name 'VQE_experiment_ENERGY' is not defined

In [26]:
INPUT = np.array([4.52776195, 4.42573835, 1.72722763, 3.14817708, 0.28249851,
       1.43945444, 3.90949061, 7.84202741, 4.38961051, 6.35661061,
       3.28588646, 6.25964932])

opt_params, list_of_inputs, list_of_outputs = Adam_Opt(INPUT, LinAlgEnergy,
                                                       calc_gradient_ADAM_linAlg, 
                                                       learning_rate=0.01, 
                                                       beta_1=0.9, 
                                                       beta_2=0.999, 
                                                       epsilon=1e-8,
                                                       max_iter=50,
                                                       disp=True,
                                                       tolerance=1e-5,
                                                       store_steps=True)

LinAlgEnergy(opt_params)

step: 1 input:[4.53776192 4.41573838 1.71722782 3.15817707 0.29249851 1.44945444
 3.89949096 7.85202739 4.39961051 6.34661061 3.29588646 6.26964932] obj_funct: -7.860379018987887
step: 2 input:[4.5321305  4.41095965 1.70735166 3.16566125 0.30194262 1.45941703
 3.88979323 7.86196076 4.4088644  6.33668667 3.3039254  6.27848309] obj_funct: -7.861668889844979
step: 3 input:[4.5251527  4.4135399  1.69765185 3.16742469 0.30991516 1.46930319
 3.87996428 7.87193541 4.41590445 6.32701327 3.31048037 6.2824556 ] obj_funct: -7.862290028263764
step: 4 input:[4.51899716 4.41923698 1.68806363 3.16562202 0.31527341 1.47908006
 3.87011379 7.881903   4.41923589 6.3178173  3.31605543 6.28166537] obj_funct: -7.862668862429345
step: 5 input:[4.51510619 4.42634727 1.67850892 3.16242746 0.31764255 1.48871596
 3.8605616  7.89151935 4.41908208 6.30936837 3.32097765 6.27822981] obj_funct: -7.863008733846836
step: 6 input:[4.5134173  4.4341431  1.66897045 3.15946833 0.31754038 1.49817604
 3.85165408 7.90014506 4

step: 47 input:[4.55191917 4.54682851 1.55820949 3.16125566 0.30007676 1.54831077
 3.83203538 7.89653322 4.40987653 6.29885122 3.29233383 6.29398481] obj_funct: -7.864339914677057
step: 48 input:[4.55284793 4.54918696 1.55902261 3.16090242 0.30036582 1.54886148
 3.83123382 7.89670552 4.41050966 6.29907873 3.29154226 6.29423311] obj_funct: -7.864343254842738
step: 49 input:[4.5534777  4.55170116 1.55987589 3.16043732 0.30077164 1.54954667
 3.83061631 7.89670898 4.41120087 6.2990495  3.29086355 6.29423137] obj_funct: -7.864347410789735
step: 50 input:[4.55383355 4.55428763 1.56073891 3.16001904 0.30119385 1.55034305
 3.83016092 7.89648636 4.41184851 6.29877879 3.2903192  6.29409434] obj_funct: -7.8643513034711665


-7.8643513034711665

In [28]:
INPUT = np.array([4.55383355, 4.55428763, 1.56073891, 3.16001904, 0.30119385,
       1.55034305, 3.83016092, 7.89648636, 4.41184851, 6.29877879,
       3.2903192 , 6.29409434])

opt_params, list_of_inputs, list_of_outputs = Adam_Opt(INPUT, LinAlgEnergy,
                                                       calc_gradient_ADAM_linAlg, 
                                                       learning_rate=0.1, 
                                                       beta_1=0.9, 
                                                       beta_2=0.999, 
                                                       epsilon=1e-8,
                                                       max_iter=50,
                                                       disp=True,
                                                       tolerance=1e-5,
                                                       store_steps=True)

LinAlgEnergy(opt_params)

step: 1 input:[4.45383651 4.65428541 1.66063066 3.26001691 0.20119558 1.65034253
 3.9301025  7.79648798 4.31184945 6.19877899 3.39031781 6.19410008] obj_funct: -7.8161512298755005
step: 2 input:[4.52674417 4.5805509  1.73858357 3.18580443 0.27540659 1.58002943
 3.85604658 7.87019111 4.38595888 6.27063974 3.46813212 6.2684148 ] obj_funct: -7.847103900196265
step: 3 input:[4.46804869 4.52170166 1.76111351 3.11397839 0.34550899 1.50853427
 3.79426456 7.93937452 4.45736763 6.34132142 3.4088386  6.33935341] obj_funct: -7.8532689052586955
step: 4 input:[4.40793242 4.51470315 1.81849087 3.08146432 0.36736151 1.48545993
 3.73802447 7.9763145  4.48706781 6.39150188 3.34657162 6.35658553] obj_funct: -7.836674449840366
step: 5 input:[4.38424283 4.54278412 1.886297   3.08725006 0.35363563 1.50178348
 3.67536492 7.98361051 4.47990545 6.41451213 3.32250704 6.33554282] obj_funct: -7.836485138728168
step: 6 input:[4.39453117 4.58464457 1.9638765  3.11525595 0.32364205 1.53705807
 3.6063844  7.97118516

step: 47 input:[4.40692923 4.71313198 1.67854709 3.16251425 0.29940339 1.56949421
 3.35316952 7.86818927 4.40522255 6.3384203  3.43749632 6.31280493] obj_funct: -7.864272753488862
step: 48 input:[4.40415511 4.70660344 1.67804462 3.1550689  0.30110849 1.56748209
 3.34988483 7.8714298  4.40626786 6.33562804 3.43594939 6.316955  ] obj_funct: -7.864349145022306
step: 49 input:[4.40209181 4.70113971 1.67765597 3.14806783 0.30339569 1.56304499
 3.34784975 7.87324867 4.40828781 6.33469506 3.43449429 6.32005569] obj_funct: -7.86429875517673
step: 50 input:[4.40174885 4.69850169 1.67746119 3.14344    0.30509073 1.55767856
 3.34652284 7.87295927 4.41027286 6.33593712 3.43393287 6.32064581] obj_funct: -7.864230983925799


-7.864230983925799

In [29]:
Hamilt.molecule.ccsd_energy

array(-7.88097349)